In [1]:
import pandas as pd
import numpy as np
import json
import re


%matplotlib inline
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=12)
mpl.rc('xtick', labelsize=8)
mpl.rc('ytick', labelsize=8)


matplotlib.rc('font', family='AppleGothic')  # MacOS
plt.rcParams['axes.unicode_minus'] = False


# pandas max row 
pd.options.display.max_rows = 4000


In [2]:

b_unit = pd.read_csv('data/1.세종시_전유부.csv', dtype={'로트':object})
b_info = pd.read_csv('data/2.세종시_표제부.csv',  dtype={'로트':object})

/Users/seungjun/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## 표제부 정리
- 동명칭, 특징 정리  
예) 101동, 유치원, 경비실

In [3]:
b_info.head()

,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,동명칭,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일,Unnamed: 15
0,1,세종특별자치시 반곡동 146-75번지,36110,10100,1,146,75,일반,NaN,NaN,135.24,벽돌구조,단독주택,0,1977.0,NaN
1,2,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,1동,576,강파이프구조,동.식물관련시설,0,1999.0,NaN
2,3,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,2동,336,강파이프구조,동.식물관련시설,0,20041223.0,NaN
3,4,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,3동,127.19,강파이프구조,창고시설,0,19950726.0,NaN
4,5,세종특별자치시 반곡동 180-1번지,36110,10100,1,180,1,일반,솔빛숲유치원,NaN,2697.74,철근콘크리트구조,교육연구시설,0,20190228.0,NaN


In [4]:
b_info.dtypes

순번               int64
대지위치            object
시군구코드            int64
법정동코드            int64
대지구분코드           int64
번                int64
지                int64
대장구분코드명         object
건물명             object
동명칭             object
연면적(㎡)          object
구조코드명           object
주용도코드명          object
세대수(세대)         object
사용승인일          float64
Unnamed: 15    float64
dtype: object

In [5]:
# 표제부 동명칭 
dong_name = b_info[['동명칭']]
dong_names = b_info['동명칭'].unique()


### 패턴 매칭
- (숫자)동


In [6]:
# 숫자 동 분리
b_info['동'] = b_info['동명칭'].str.findall('[\d]+동').str.join("")
# 숫자 ~ 동 제거
b_info['동명칭'] = b_info['동명칭'].str.findall('[^\d동()-]').str.join("")


In [7]:
b_info.head(3)

,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,동명칭,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일,Unnamed: 15,동
0,1,세종특별자치시 반곡동 146-75번지,36110,10100,1,146,75,일반,NaN,NaN,135.24,벽돌구조,단독주택,0,1977.0,NaN,NaN
1,2,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,,576,강파이프구조,동.식물관련시설,0,1999.0,NaN,1동
2,3,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,,336,강파이프구조,동.식물관련시설,0,20041223.0,NaN,2동


In [8]:
b_info.to_csv('test.csv', encoding='utf-8-sig')

## 전유부 Refactoring
공모전에서 제공한 데이터가 부족하여 공공데이터 API를 활용하여 전유부 데이터를 추가

In [22]:
# API를 통해 생생된 전유부 Data 가져오기
b_unit = pd.read_csv('전유부.csv')

In [26]:
# Covert Data Types
b_unit['시군구코드'] = b_unit['시군구코드'].astype(object)
b_unit['법정동코드'] = b_unit['법정동코드'].astype(object)
b_unit['층번호'] = b_unit['층번호'].astype(np.int64)
b_unit['번'] = b_unit['번'].astype(object)
b_unit['지'] = b_unit['지'].astype(object)
b_unit['층구분코드'] = b_unit['층구분코드'].astype(object)

In [27]:
b_unit.dtypes

대지주소         object
시군구코드        object
법정동코드        object
동명칭          object
호명칭          object
층구분코드        object
층번호           int64
면적(전용면적)    float64
건물명          object
번            object
지            object
주용도코드        object
dtype: object

In [29]:
b_unit.head(10)

,대지주소,시군구코드,법정동코드,동명칭,호명칭,층구분코드,층번호,면적(전용면적),건물명,번,지,주용도코드
0,세종특별자치시 반곡동 4121-1002번지,36110,10100,주건축물제1동,101,20,1,3.2402,세종펠리스,4121,1002,일반음식점
1,세종특별자치시 반곡동 4121-1002번지,36110,10100,주건축물제1동,101,20,1,39.3700,세종펠리스,4121,1002,일반음식점
2,세종특별자치시 반곡동 4121-1002번지,36110,10100,주건축물제1동,101,20,1,10.3346,세종펠리스,4121,1002,일반음식점
3,세종특별자치시 반곡동 4121-1002번지,36110,10100,주건축물제1동,101,20,1,33.6765,세종펠리스,4121,1002,일반음식점
4,세종특별자치시 반곡동 4121-1002번지,36110,10100,주건축물제1동,102,20,1,48.9500,세종펠리스,4121,1002,일반음식점
5,세종특별자치시 반곡동 4121-1002번지,36110,10100,주건축물제1동,102,20,1,12.8493,세종펠리스,4121,1002,일반음식점
6,세종특별자치시 반곡동 4121-1002번지,36110,10100,주건축물제1동,102,20,1,4.0286,세종펠리스,4121,1002,일반음식점
7,세종특별자치시 반곡동 4121-1002번지,36110,10100,주건축물제1동,102,20,1,41.8710,세종펠리스,4121,1002,일반음식점
8,세종특별자치시 반곡동 4121-1002번지,36110,10100,주건축물제1동,104,20,1,38.2000,세종펠리스,4121,1002,일반음식점
9,세종특별자치시 반곡동 4121-1002번지,36110,10100,주건축물제1동,104,20,1,10.0274,세종펠리스,4121,1002,일반음식점
